<a href="https://colab.research.google.com/github/hbhb0311/LikelionStudy/blob/main/4.NLP/3.TextSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### n-gram 유사도



In [ ]:
!pip install konlpy

In [2]:
from konlpy.tag import Komoran

# 실험 문장 정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다.'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다.'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

In [3]:
# 형태소 분석기로 명사 추출
komoran = Komoran()

bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

In [4]:
# n-gram을 해주는 함수 
def word_ngram(bow, num_gram):
  text = tuple(bow)
  ngrams = [text[i : i+num_gram] for i in range(len(text))]
  return ngrams

In [5]:
# 함수 이용해 n-gram 토큰 추출
doc1 = word_ngram(bow1, 2)
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

In [6]:
# 내가 만든 코드 : 길이가 다를 때 오류 발생
def similarity(doc1, doc2):
  cnt = 0
  for i in range(len(doc1)):
    if doc1[i] == doc2[i]:
      cnt +=1
  
  return cnt / len(doc1)

In [7]:
def similarity(doc1, doc2):
  cnt = 0
  for token in doc1:
    if token in doc2:
      cnt += 1
  
  return cnt / len(doc1)

In [8]:
# 유사도 계산
r1 = similarity(doc1, doc2)
r2 = similarity(doc3, doc1)

print(r1)
print(r2)

0.6666666666666666
0.0


### 코사인 유사도

In [9]:
from konlpy.tag import Komoran
import numpy as np

In [11]:
def cos_sim(x, y):
  return np.dot(x,y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [12]:
# 실험 문장 정의
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다.'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다.'
sentence3 = '나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다.'

# 형태소 분석기로 명사 추출
komoran = Komoran()

bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

# 단어 묶음 리스트를 하나로 합침
bow = bow1 + bow2 + bow3
print(bow)

['6월', '뉴턴', '선생님', '제안', '트리니티', '입학', '6월', '뉴턴', '선생님', '제안', '대학교', '입학', '밥', '뉴턴', '선생', '님과 함께']


In [13]:
# 단어 묶음에서 중복 제거해 단어 사전 구축
word_dics = []

for token in bow:
  if token not in word_dics:
    word_dics.append(token)

In [14]:
word_dics

['6월', '뉴턴', '선생님', '제안', '트리니티', '입학', '대학교', '밥', '선생', '님과 함께']

In [15]:
# TDM 만드는 함수
def make_term_doc_mat(sentence_bow, word_dics):
  freq_mat = {}

  for word in word_dics:
    freq_mat[word] = 0
  
  for word in word_dics:
    if word in sentence_bow:
      freq_mat[word] += 1

  return freq_mat

In [16]:
# 문장별 단어 빈도 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)

print(freq_list1)
print(freq_list2)
print(freq_list3)

{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1, '선생': 1, '님과 함께': 1}


In [17]:
# 단어 벡터 만드는 함수
def make_vector(tdm):
  vec = []
  for key in tdm:
    vec.append(tdm[key])
  
  return vec

In [18]:
# 문장 벡터 생성
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))

In [19]:
# 코사인 유사도 계산
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc3, doc1)

print(r1)
print(r2)

0.8333333333333335
0.20412414523193154
